In [1]:
import sys
from os import getenv
import warnings
warnings.filterwarnings("ignore")

GONZAG_DIR   = getenv('HOME')+'/DEV/gonzag'
sys.path.append(GONZAG_DIR)

import gonzag as gzg
from gonzag.config import ldebug

GONZAG_DATA_DIR = '/Users/auraoupa/Work/git/gonzag'

# Satellite input data:
name_sat = 'Sentinel3A'
file_sat = GONZAG_DATA_DIR+'/zarr_SENTINEL3A_20170130-20170303'
name_ssh_sat = 'sla_unfiltered'

# Model input data:
name_mod = 'eNATL60-WestMed'
file_mod = GONZAG_DATA_DIR+'/zarr_sossheig_box_WestMed_eNATL60-BLBT02_20170201-20170228'
name_ssh_mod = 'sossheig'
file_lsm_mod = file_mod; name_lsm_mod = '_FillValue' ; # we use _FillValue attribute of "nams_ssh_mod" in "file_mod"
l_griddist = False ; # grid is not strongly distorded





In [2]:
    (it1,it2), (Nts,Ntm) = gzg.GetEpochTimeOverlap( file_sat , file_mod )


In [3]:
print(it1,it2)

2017-02-01T00:30:00.000000000 2017-02-28T23:30:00.000000000


In [4]:
print(Nts,Ntm)

1599082 672


In [5]:
ncfile_sat, ncfile_mod = file_sat , file_mod


In [6]:
from sys import exit
from math import radians, cos, sin, asin, sqrt, pi, tan, log, atan2, copysign
import numpy as nmp
from gonzag.config import ldebug, R_eq, R_pl, deg2km
IsZarr=True

In [7]:
    if not IsZarr:
        from gonzag.ncio import GetTimeInfo
#        print('import nc')
    else:
        from gonzag.zarrio import GetTimeInfo
#        print('import zarr')
    #
    nts, range_sat = GetTimeInfo( ncfile_sat )
    ntm, range_mod = GetTimeInfo( ncfile_mod )
    #
    (zt1_s,zt2_s) = range_sat
    (zt1_m,zt2_m) = range_mod
    if ldebug: print('\n *** [GetEpochTimeOverlap()] Earliest/latest dates:\n   => for satellite data:',zt1_s,zt2_s,'\n   => for model     data:',zt1_m,zt2_m,'\n')
    if (zt1_m >= zt2_s) or (zt1_s >= zt2_m) or (zt2_m <= zt1_s) or (zt2_s <= zt1_m):
        MsgExit('No time overlap for Model and Track file')


In [8]:
print((max(zt1_s, zt1_m), min(zt2_s, zt2_m)), (nts, ntm))

(numpy.datetime64('2017-02-01T00:30:00.000000000'), numpy.datetime64('2017-02-28T23:30:00.000000000')) (1599082, 672)


In [9]:
print(zt1_m)

2017-02-01T00:30:00.000000000


In [10]:
import numpy as nmp
import xarray as xr
from calendar import timegm
from datetime import datetime as dtm
from gonzag.config import ldebug, ivrb, rmissval
from gonzag.utils  import MsgExit


In [11]:
ncfile=ncfile_sat

In [12]:
    if ldebug: print(' *** [GetTimeInfo()] Getting calendar/time info in '+ncfile+' ...')
    id_f = xr.open_zarr(ncfile)
    for cd in [ 'time', 'time_counter', 'TIME', 'record', 't', 'none' ]:
        if cd in id_f.coords: break
    if cd == 'none': MsgExit('found no time-record dimension in file '+ncfile)
    if ldebug: print('   => time/record dimension is "'+cd+'"')
    nt = id_f[cd].size
    clndr = id_f[cd]
    dt1 = clndr[0]
    dt2 = clndr[nt-1]
    id_f.close()


In [13]:
id_f = xr.open_zarr(ncfile,decode_cf=False)


In [14]:
cd

'time'

In [15]:
id_f[cd]

<xarray.DataArray 'time' (time: 1599082)>
array([24500.989341, 24500.989352, 24500.989364, ..., 24534.993906,
       24534.993918, 24534.99393 ])
Coordinates:
  * time     (time) float64 2.45e+04 2.45e+04 2.45e+04 ... 2.453e+04 2.453e+04
Attributes:
    axis:           T
    calendar:       gregorian
    long_name:      Time of measurement
    standard_name:  time
    units:          days since 1950-01-01 00:00:00
    _FillValue:     nan

In [16]:
print(nt, (dt1.values,dt2.values))

1599082 (numpy.datetime64('2017-01-29T23:44:39.027517696'), numpy.datetime64('2017-03-04T23:51:15.516345088'))


In [17]:
dt1.values

numpy.datetime64('2017-01-29T23:44:39.027517696')

In [18]:
def ToEpochTime( vt, units, calendar ):
    '''
    # INPUT:
    #   * vt: time vector provided as something like: "days since ..."
    #   * units, calendar:
    #
    # OUTPUT:
    #   * vet: time vector converted to UNIX epoch time,
    #          aka "seconds since 1970-01-01 00:00:00"
    #          => as FLOAT !! not INTEGER !!!
    '''
    cfrmt = '%Y-%m-%d %H:%M:%S'
    #
    lvect = not (nmp.shape(vt)==())
    #
    if lvect:
        nt = len(vt)
        t0 = vt[0]
    else:
        t0 = vt
    if ivrb>0: print(' *** [ToEpochTime()]: original t0 as "'+units+'" => ', t0)
    t0d = num2date( t0, units, calendar )
    if ivrb>0: print(' *** [ToEpochTime()]: intitial date in datetime format => ', t0d)

    # We need to round this to the nearest second, because our target format is Epoch time (seconds since 1970)
    # and we want an integer!
    rdec = t0d.microsecond*1.E-6
    # t0 as "float" UNIX time:
    t0E = float( timegm( dtm.strptime( t0d.strftime(cfrmt) , cfrmt ).timetuple() ) + rdec )

    if lvect:
        # we are not going to convert the whole array but instead:
        if   units[0:10] == 'days since':
            vdt = (vt[1:] - vt[0])*86400.
        elif units[0:11] == 'hours since':
            vdt = (vt[1:] - vt[0])*3600.
        elif units[0:13] == 'seconds since':
            vdt =  vt[1:] - vt[0]
        else:
            MsgExit('[ToEpochTime()] => unknown time unit: '+units)
        vet = nmp.zeros(nt)
        vet[0]  = t0E
        vet[1:] = t0E + vdt[:]
        del vdt
    else:
        vet = t0E
    #
    return vet


In [31]:
from netCDF4 import num2date, default_fillvals


In [32]:
    if ldebug: print(' *** [GetTimeInfo()] Getting calendar/time info in '+ncfile+' ...')
    id_f = xr.open_zarr(ncfile,decode_cf=False)
    for cd in [ 'time', 'time_counter', 'TIME', 'record', 't', 'none' ]:
        if cd in id_f.coords: break
    if cd == 'none': MsgExit('found no time-record dimension in file '+ncfile)
    if ldebug: print('   => time/record dimension is "'+cd+'"')
    nt = id_f[cd].size    
    clndr = id_f[cd]
    rt1 = ToEpochTime( clndr[0],    clndr.units, clndr.calendar )
    rt2 = ToEpochTime( clndr[nt-1], clndr.units, clndr.calendar )    


In [30]:
clndr.attrs['units']

'days since 1950-01-01 00:00:00'

In [34]:
print(rt2)

1488671475.516345
